In [2]:
import os
import json
import logging
from json import JSONDecodeError

import wikipedia
from datasets import load_from_disk

In [4]:
logging.basicConfig(filename='wiki.log', encoding='utf-8', level=logging.ERROR)

In [5]:
root_dir = r"C:\Users\onurg\.cache\huggingface\datasets"
data_dir = "updated_wiki40b"
#language = "filtered_small"
language = "long_small_en"

path = os.path.join(root_dir, data_dir, language)
dataset = load_from_disk(path)

In [6]:
col_names = set(dataset.column_names)
cols_to_remove = col_names - set(("title", "wikidata_id"))

dataset = dataset.remove_columns(cols_to_remove)

In [7]:
title_dict = {}

def title_id_matcher(example):
    title_dict[example["wikidata_id"]] = example["title"]

dataset.map(title_id_matcher)

100%|██████████| 639046/639046 [01:02<00:00, 10246.08ex/s]


Dataset({
    features: ['title', 'wikidata_id'],
    num_rows: 639046
})

## Continue

In [28]:
stopped_file = "titles.json"
logfile_name = "wiki.log"

# Set of IDs that are already stored in json file
data = []
with open(stopped_file, encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line)["id"])
seen_ids = set(data)

# Set of IDs that are already logged to cause error
with open(logfile_name) as f:
    lines = f.readlines()
logged_ids = {line.split(",")[1].strip() for line in lines}

# Union of the respective sets
seen_ids = seen_ids.union(logged_ids)

title_dict = {k: v for k, v in title_dict.items() if k not in seen_ids}

In [ ]:
json_name = "titles.json"

with open(json_name, 'a') as f:
    for wiki_id, title in title_dict.items():
        dict = {}
        dict["id"] = wiki_id
        dict["title"] = title

        try:
            p = wikipedia.page(title, auto_suggest=False)
            categories = "#".join(p.categories)

        except wikipedia.DisambiguationError:
            logging.error('DisambiguationError, %s, %s', wiki_id, title)
            continue

        except wikipedia.PageError:
            logging.error('PageError, %s, %s', wiki_id, title)
            continue

        except KeyError:
            logging.error('KeyError, %s, %s', wiki_id, title)
            continue
        
        except JSONDecodeError:
            logging.error('JSONDecodeError, %s, %s', wiki_id, title)
            continue

        
        dict["categories"] = categories
        f.write(json.dumps(dict) + "\n")